### The Modeling Problem

We assume that DonorsChoose has hired a digital content expert who will review projects and help teachers improve their postings and increase their chances of reaching their funding threshold. Because this individualized review is a labor-intensive process, the digital content expert has ** time to review and support only 10% of the projects posted to the platform on a given day**. 

Our task is to help this content expert focus their limited resources on projects that most need the help. As such, we want to build a model to **identify projects that are least likely to be fully funded before they expire**  and pass them off to the digital content expert for review.


# Getting Set Up

In [8]:
!mkdir /Users

mkdir: /Users: File exists


In [9]:
!pip install kaggle

!mkdir /Users/.kaggle
!touch /Users/.kaggle/kaggle.json
api_token = {"username":"ploped123","key":"eeeeba8fc52706723e4c1bcf41ae6fd3"}

import json
import zipfile
import os
with open('/Users/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /Users/.kaggle/kaggle.json

!kaggle competitions download -c kdd-cup-2014-predicting-excitement-at-donors-choose


[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: pip install --upgrade pip
mkdir: /Users/.kaggle: Permission denied
touch: /Users/.kaggle/kaggle.json: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: '/Users/.kaggle/kaggle.json'

In [10]:
import zipfile

z= zipfile.ZipFile('kdd-cup-2014-predicting-excitement-at-donors-choose.zip')
z.extractall("/")

FileNotFoundError: [Errno 2] No such file or directory: 'kdd-cup-2014-predicting-excitement-at-donors-choose.zip'

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
donations_df = pd.read_csv("donations.csv.zip")
projects_df = pd.read_csv("projects.csv.zip")

# Getting the Baseline Rate

In [ ]:
from datetime import timedelta
donations_df['donation_timestamp'] = pd.to_datetime(donations_df.donation_timestamp)
projects_df['date_posted'] = pd.to_datetime(projects_df.date_posted)

a = pd.merge(projects_df, donations_df, on=['projectid'], how='left')
a['in_4_months'] = (a['donation_timestamp'] - a['date_posted']) < timedelta(days=120)

values = {"donation_to_project": 0.0}
a = a.fillna(value=values)
a.loc[a['in_4_months'] == False, 'donation_to_project'] = 0.0

donation_in_4_months = a.groupby(['projectid'])['donation_to_project'].sum().reset_index(name='donation_in_4_months')
df = pd.merge(projects_df, donation_in_4_months, on=['projectid'], how='left')
df['if_fully_funded_after_4_months'] = df['total_price_excluding_optional_support'] <= df['donation_in_4_months']
df['if_fully_funded_after_4_months'].value_counts()

In [ ]:
ratio = df['if_fully_funded_after_4_months'].value_counts()[1] / (df['if_fully_funded_after_4_months'].value_counts()[0] + df['if_fully_funded_after_4_months'].value_counts()[1])
print("Percentage of projects that are fully funded is: %.2f" % (ratio*100) + "%")